In [ ]:
import io
import os
import tempfile
from pathlib import Path

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models.morphology import (
    BrainLocation,
    BrainRegion,
    ReconstructionMorphology,
    Species,
    Strain,
)

### Initialize the entitycore client

In [ ]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="103d7868-147e-4f07-af0d-71d8568f575c",
    project_id="103d7868-147e-4f07-af0d-71d8568f575c",
)
client = Client(api_url=entitycore_api_url, project_context=project_context)
token = os.getenv("ACCESS_TOKEN", "XXX")

## Search for entities

In [ ]:
species = list(
    client.search(entity_type=Species, query={"name": "Mus musculus"}, token=token, limit=10)
)[0]
rprint(species)

strain = list(client.search(entity_type=Strain, query={"name": "Cux2-CreERT2"}, token=token))[0]
rprint(strain)

### Create a morphology object

In [ ]:
# No GET endpoint yet for brain-region

brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)
brain_region = BrainRegion(
    id=68,
    name="Frontal pole, layer 1",
    acronym="FRP1",
    children=[],
)
morphology = ReconstructionMorphology(
    name="my-morph",
    description="A morphology",
    species=species,
    strain=strain,
    brain_region=brain_region,
    location=brain_location,
    legacy_id="temp",
)
rprint(morphology)

## Register morphology and upload assets

In [ ]:
registered = client.register(entity=morphology, token=token)
rprint(registered)


with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "morph.h5")
    file1.write_text("h5")
    file1.touch()

    file2 = Path(tdir, "morph.swc")
    file2.write_text("swc")
    file2.touch()

    # use a filepath to register first asset
    asset1 = client.upload_file(
        entity_id=registered.id,
        entity_type=ReconstructionMorphology,
        file_path=file1,
        file_content_type="application/h5",
        token=token,
    )
    rprint(asset1)

    # use an in-memory buffer to upload second asset
    buffer = io.BytesIO(b"morph bytes buffer")

    asset2 = client.upload_content(
        entity_id=registered.id,
        entity_type=ReconstructionMorphology,
        file_content=buffer,
        file_name="buffer.h5",
        file_content_type="application/swc",
        token=token,
    )
    rprint(asset2)
    morphology = morphology.evolve(assets=[asset1, asset2])

## Retrieve it

In [ ]:
fetched = client.get(entity_id=registered.id, entity_type=ReconstructionMorphology, token=token)
rprint(fetched)
fetched_wout_assets = client.get(
    entity_id=registered.id, entity_type=ReconstructionMorphology, token=token, with_assets=False
)
rprint(fetched_wout_assets)

## Search it

In [ ]:
it = client.search(
    entity_type=ReconstructionMorphology,
    query={"name": "my-morph", "page": 1, "page_size": 2},
    token=token,
    limit=0,
)
hits = list(it)

print(len(hits))
rprint(hits[0])